# Load, Aggregate And Clean Complaints Data

In [1]:
import json
from glob import glob
import pandas as pd
import numpy as np
import itertools

from loguru import logger
from IPython.display import display, clear_output

## Path to complaints dataset

In [19]:
complaints_dataset = "../Raw_Data/Raw_API/Complaints/Complaints"
output_path = '../Processed_Data/API/Complaints.pkl'

## Aggregate all json file in this directory.

In [17]:
files = glob(complaints_dataset+'/*.json')
len(files)

6540

In [4]:
data = []

for i, file in enumerate(files):
    
    clear_output()
    logger.info((i, file))
    with open(file, 'r') as f: data.append(json.load(f))
        
df = pd.DataFrame(itertools.chain.from_iterable(data))
df.shape

2022-04-26 19:08:04.848 | INFO     | __main__:<module>:6 - (6539, '../Raw_Data/Raw_API/Complaints/Complaints/[5532]2017_MINI_COOPER S.json')


(750742, 12)

In [5]:
df

,components,crash,dateComplaintFiled,dateOfIncident,fire,manufacturer,numberOfDeaths,numberOfInjuries,odiNumber,products,summary,vin
0,"STRUCTURE,ENGINE",False,10/30/2019,10/23/2019,False,Mazda Motor Corp.,0,0,11277034,"[{'manufacturer': 'Mazda Motor Corp.', 'produc...",TL* THE CONTACT OWNS A 2010 MAZDA 3. WHILE DRI...,JM1BL1SF581
1,"POWER TRAIN,ELECTRICAL SYSTEM,AIR BAGS",False,04/14/2019,11/21/2018,False,Mazda Motor Corp.,0,0,11196097,"[{'manufacturer': 'Mazda Motor Corp.', 'produc...",BLINKING AIR BAG INDICATOR LIGHT SHORTLY AFTER...,JM1BL1H45A1
2,EXTERIOR LIGHTING,False,09/02/2018,03/03/2018,False,Mazda Motor Corp.,0,0,11124015,"[{'manufacturer': 'Mazda Motor Corp.', 'produc...",WATER INTERMITTENTLY LEAKS INTO PASSENGER SIDE...,JM1BL1H34A1
3,"SUSPENSION,AIR BAGS",False,08/16/2017,08/04/2015,False,Mazda Motor Corp.,0,1,11015673,"[{'manufacturer': 'Mazda Motor Corp.', 'produc...",TL* THE CONTACT OWNS A 2010 MAZDA 3. WHILE DRI...,JM1BL1H34A1
4,SEATS,False,02/15/2017,02/15/2017,False,Mazda Motor Corp.,0,0,10954913,"[{'manufacturer': 'Mazda Motor Corp.', 'produc...","WHILE DRIVING, THE SWING ARM WHICH ATTACHES TH...",JM1BL1H43A1
...,...,...,...,...,...,...,...,...,...,...,...,...
750737,"SERVICE BRAKES, HYDRAULIC",False,06/18/2007,01/01/1997,False,"General Motors, LLC",0,0,10193617,"[{'manufacturer': 'General Motors, LLC', 'prod...","FRONT BRAKES WEARING OUT EVERY 6,000 MI, REAR ...",
750738,"SERVICE BRAKES, HYDRAULIC,ENGINE AND ENGINE CO...",False,11/07/2006,05/20/1999,False,"General Motors, LLC",0,0,10172825,"[{'manufacturer': 'General Motors, LLC', 'prod...",ANTI LOCK BRAKES PULSATE AT LOW SPEED ON DRY P...,
750739,STEERING,False,08/15/2006,01/01/2006,False,"General Motors, LLC",0,0,10165589,"[{'manufacturer': 'General Motors, LLC', 'prod...",EXTREME OVER STEERING AT SPEEDS OVER 45 MPH. ...,
750740,"VEHICLE SPEED CONTROL,ENGINE",False,05/24/2019,05/23/2019,False,"BMW of North America, LLC",0,0,11209703,"[{'manufacturer': 'BMW of North America, LLC',...",TL* THE CONTACT OWNS A 2017 MINI COOPER S. WHI...,KMHTC6AE1DU


## Drop some unused columns

In [6]:
df.drop(['odiNumber','summary','vin','dateOfIncident','manufacturer'], axis=1, inplace=True)
df.head()

,components,crash,dateComplaintFiled,fire,numberOfDeaths,numberOfInjuries,products
0,"STRUCTURE,ENGINE",False,10/30/2019,False,0,0,"[{'manufacturer': 'Mazda Motor Corp.', 'produc..."
1,"POWER TRAIN,ELECTRICAL SYSTEM,AIR BAGS",False,04/14/2019,False,0,0,"[{'manufacturer': 'Mazda Motor Corp.', 'produc..."
2,EXTERIOR LIGHTING,False,09/02/2018,False,0,0,"[{'manufacturer': 'Mazda Motor Corp.', 'produc..."
3,"SUSPENSION,AIR BAGS",False,08/16/2017,False,0,1,"[{'manufacturer': 'Mazda Motor Corp.', 'produc..."
4,SEATS,False,02/15/2017,False,0,0,"[{'manufacturer': 'Mazda Motor Corp.', 'produc..."


In [ ]:
df_clean =  pd.concat([pd.json_normalize(df.products)[0].apply(pd.Series),
           df],axis=1)
df_clean.group('type').size()

In [ ]:
df_clean.drop([['products','type']], axis=1, inplace=True)
df_clean.head()

## Convert date columns to Dateformat

In [8]:
df_clean['dateComplaintFiled'] = pd.to_datetime(df_clean['dateComplaintFiled'], format='%m/%d/%Y').dt.strftime("%Y-%m-%d")
df_clean.head()

,manufacturer,productMake,productModel,productYear,type,components,crash,dateComplaintFiled,fire,numberOfDeaths,numberOfInjuries
0,Mazda Motor Corp.,MAZDA,MAZDASPEED3,2010,Vehicle,"STRUCTURE,ENGINE",False,2019-10-30,False,0,0
1,Mazda Motor Corp.,MAZDA,MAZDASPEED3,2010,Vehicle,"POWER TRAIN,ELECTRICAL SYSTEM,AIR BAGS",False,2019-04-14,False,0,0
2,Mazda Motor Corp.,MAZDA,MAZDASPEED3,2010,Vehicle,EXTERIOR LIGHTING,False,2018-09-02,False,0,0
3,Mazda Motor Corp.,MAZDA,MAZDASPEED3,2010,Vehicle,"SUSPENSION,AIR BAGS",False,2017-08-16,False,0,1
4,Mazda Motor Corp.,MAZDA,MAZDASPEED3,2010,Vehicle,SEATS,False,2017-02-15,False,0,0


## Clean components columns

In [21]:
def clean_components(list_):
    list_ = list_.replace(', ', ',')
    list_ = list_.replace(' ,', ',')
    return list_
df_clean['components'] = df_clean['components'].apply(clean_components)

## Output cleaned version

In [23]:
df_clean.to_pickle(output_path) 